# Figo Prasetyo Nugroho | figo

In [26]:
!pip install langchain
!pip install langchain-groq

import os
import json
import requests
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [27]:
import getpass

SECTORS_API_KEY=getpass.getpass(prompt='Insert SECTORS_API_KEY : ')

Insert SECTORS_API_KEY : ········


In [28]:
GROQ_API_KEY=getpass.getpass(prompt='Insert GROQ_API_KEY : ')

Insert GROQ_API_KEY : ········


In [29]:
import requests
import pprint

headers = {
    "Authorization": SECTORS_API_KEY
}


def get_overview(stock:str, section:str) -> requests:
    url = f"https://api.sectors.app/v1/company/report/{stock}/?sections={section}"
    response = requests.get(url, headers=headers)
    return response


response = get_overview("BBRI", "financials")
pprint.pprint(response.json())

{'company_name': 'PT Bank Rakyat Indonesia (Persero) Tbk',
 'financials': {'cash_flow_debt_ratio': 0.7816831005258889,
                'dar_mrq': 0.0753361231980293,
                'der_mrq': 0.501336568777284,
                'historical_financials': [{'cash_and_equivalents': None,
                                           'cash_only': None,
                                           'current_liabilities': None,
                                           'earnings': 32351133000000,
                                           'earnings_before_tax': 41753694000000,
                                           'fixed_assets': None,
                                           'gross_profit': None,
                                           'operating_pnl': 35523560000000,
                                           'revenue': 98729001000000,
                                           'tax': 9335208000000,
                                           'total_assets': 1296898292000000,
          

In [30]:
from langchain_core.tools import tool

def retrieve_from_endpoint(url: str) -> dict:
    headers = {"Authorization": SECTORS_API_KEY}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)

def get_today_date() -> str:
    """
    Get today's date
    """
    from datetime import date

    today = date.today()
    return today.strftime("%Y-%m-%d")

@tool
def get_top_companies_by_tx_volume(
    start_date: str, end_date: str, top_n: int = 5
) -> str:
    """
    Get top companies by transaction volume
    """
    url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}"

    return retrieve_from_endpoint(url)

tools = [get_top_companies_by_tx_volume]
llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Answer the following queries, being as factual and analytical as you can.
            If you need the start and end dates but they are not explicitly provided,
            infer from the query. Whenever you return a list of names, return also the
            corresponding values for each name. If the volume was about a single day,
            the start and end parameter should be the same. Note that the endpoint for
            performance since IPO has only one required parameter, which is the company_name.
            Today's date is
            """
            + get_today_date() +
            """
             and if it is not specified how many are needed, by default just need only  1.
            """,
        ),
        ("human", "{input}"),
        # msg containing previous agent tool invocations and corresponding tool outputs
        MessagesPlaceholder("agent_scratchpad"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "What are the top 3 companies by transaction volume over the last 7 days?"
result = agent_executor.invoke({"input": query})
print("Answer:", "\n", result["output"], "\n\n======\n\n")




> Entering new AgentExecutor chain...

Invoking: `get_top_companies_by_tx_volume` with `{'start_date': '2024-07-24', 'end_date': '2024-07-31', 'top_n': 3}`


{"2024-07-24": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 2660984600, "price": 54}, {"symbol": "BSBK.JK", "company_name": "PT Wulandari Bangun Laksana Tbk", "volume": 1833364900, "price": 83}, {"symbol": "DOOH.JK", "company_name": "PT Era Media Sejahtera Tbk", "volume": 511754100, "price": 53}], "2024-07-25": [{"symbol": "BSBK.JK", "company_name": "PT Wulandari Bangun Laksana Tbk", "volume": 1513766400, "price": 76}, {"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 911091700, "price": 54}, {"symbol": "BBKP.JK", "company_name": "PT Bank KB Bukopin Tbk", "volume": 432596400, "price": 56}], "2024-07-26": [{"symbol": "BSBK.JK", "company_name": "PT Wulandari Bangun Laksana Tbk", "volume": 2408506000, "price": 72}, {"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek T

In [31]:
queries = [
    "What are the top 5 companies by transaction volume on the first of this month?",
    "What are the most traded stock yesterday?",
    "What are the top 7 most traded stocks between 6th June to 10th June this year?"
]

for query in queries:
    print("Question:", query)
    result = agent_executor.invoke({"input": query})
    print("Answer:", "\n", result["output"], "\n\n======\n\n")

Question: What are the top 5 companies by transaction volume on the first of this month?


> Entering new AgentExecutor chain...

Invoking: `get_top_companies_by_tx_volume` with `{'start_date': '2024-07-01', 'end_date': '2024-07-01', 'top_n': 5}`


{"2024-07-01": [{"symbol": "BRPT.JK", "company_name": "Barito Pacific Tbk", "volume": 468215600, "price": 1130}, {"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 452005800, "price": 50}, {"symbol": "TOSK.JK", "company_name": "PT Topindo Solusi Komunika Tbk.", "volume": 442612100, "price": 61}, {"symbol": "BULL.JK", "company_name": "PT Buana Lintas Lautan Tbk.", "volume": 370373200, "price": 141}, {"symbol": "TOPS.JK", "company_name": "PT Totalindo Eka Persada Tbk.", "volume": 368219800, "price": 1}]}The top 5 companies by transaction volume on the first of this month are:
1. Barito Pacific Tbk with a volume of 468,215,600.
2. PT GoTo Gojek Tokopedia Tbk with a volume of 452,005,800.
3. PT Topindo Solusi Komunika